## 預測價格

In [2]:
import os
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# --------------------------
# 1. 讀取資料
# --------------------------
df = pd.read_csv('/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv')

# --------------------------
# 2. 欄位定義
# --------------------------
cat_cols  = ['出發時段','出發機場代號','抵達時段','航空聯盟','機型分類','假期','是否為平日']
group_col = '抵達機場代號'
num_cols  = ['飛行時間_分鐘','經濟指標','機場指標','competing_flights']
log_col   = '平均價格_log'
true_col  = '平均價格'
pred_col  = '預測_平均價格'

# --------------------------
# 3. 分艙等並標記真實價格
# --------------------------
eco_raw = df[df['艙等']=='經濟艙'][cat_cols + [group_col] + num_cols + [log_col] + [true_col]].copy()
biz_raw = df[df['艙等']=='商務艙'][cat_cols + [group_col] + num_cols + [log_col] + [true_col]].copy()

# --------------------------
# 4. 編碼與標準化
# --------------------------
def encode_and_scale(df_raw):
    df_enc = pd.get_dummies(df_raw, columns=cat_cols, drop_first=True)
    scaler = StandardScaler()
    df_enc[num_cols] = scaler.fit_transform(df_enc[num_cols])
    return df_enc, scaler

eco_enc, eco_scaler = encode_and_scale(eco_raw)
biz_enc, biz_scaler = encode_and_scale(biz_raw)

# --------------------------
# 5. 取共同特徵欄位
# --------------------------
common_feats = list(set(eco_enc.columns) & set(biz_enc.columns))
for col in [log_col, true_col, group_col]:
    if col in common_feats:
        common_feats.remove(col)

# --------------------------
# 6. 訓練模型 & 拆分測試集
# --------------------------
def split_and_train(df_enc, model_type='xgb', seed=123):
    X = df_enc[common_feats]
    y = df_enc[log_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    if model_type == 'xgb':
        model = XGBRegressor(n_estimators=100, random_state=seed, n_jobs=-1)
    else:
        model = RandomForestRegressor(n_estimators=100, random_state=seed, n_jobs=-1)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"🔹[{model_type}] 測試集 MSE = {mean_squared_error(y_test, y_pred):.4f}, R2 = {r2_score(y_test, y_pred):.4f}")
    sigma2 = np.mean((y_test - y_pred)**2)
    
    return model, sigma2, df_enc.loc[X_test.index]

eco_model, sigma2_eco, eco_test = split_and_train(eco_enc, model_type='xgb')
biz_model, sigma2_biz, biz_test = split_and_train(biz_enc, model_type='rf')

# --------------------------
# 7. 真實組合與預測儲存
# --------------------------
def get_unique_combinations(df_raw, code):
    sub = df_raw[df_raw[group_col] == code]
    return sub[cat_cols + num_cols + [true_col]].drop_duplicates().reset_index(drop=True)

def predict_and_save_testset(test_df, model, scaler, sigma2, prefix):
    out_dir = 'predict_data/short'
    os.makedirs(out_dir, exist_ok=True)
    for code in sorted(test_df[group_col].unique()):
        combos = get_unique_combinations(test_df, code)
        if combos.empty:
            continue

        enc = pd.get_dummies(combos, columns=cat_cols, drop_first=True)
        enc = enc.reindex(columns=common_feats, fill_value=0)
        enc[num_cols] = scaler.transform(combos[num_cols])
        log_preds = model.predict(enc)
        combos[pred_col] = np.expm1(log_preds + 0.5 * sigma2)
        combos[group_col] = code
        combos = combos.drop_duplicates(subset=cat_cols + num_cols)
        fn = f'{out_dir}/{prefix}_{code}.csv'
        combos.to_csv(fn, index=False)
        print(f"✅ 測試集已存：{fn}（共 {len(combos)} 筆）")

# --------------------------
# 8. 執行測試集預測
# --------------------------
predict_and_save_testset(eco_raw.loc[eco_test.index], eco_model, eco_scaler, sigma2_eco, 'eco')
predict_and_save_testset(biz_raw.loc[biz_test.index], biz_model, biz_scaler, sigma2_biz, 'biz')

🔹[xgb] 測試集 MSE = 0.0546, R2 = 0.8535
🔹[rf] 測試集 MSE = 0.0092, R2 = 0.8909
✅ 測試集已存：predict_data/short/eco_BKK.csv（共 140 筆）
✅ 測試集已存：predict_data/short/eco_GMP.csv（共 21 筆）
✅ 測試集已存：predict_data/short/eco_HKG.csv（共 405 筆）
✅ 測試集已存：predict_data/short/eco_HND.csv（共 52 筆）
✅ 測試集已存：predict_data/short/eco_ICN.csv（共 102 筆）
✅ 測試集已存：predict_data/short/eco_NRT.csv（共 180 筆）
✅ 測試集已存：predict_data/short/eco_SIN.csv（共 86 筆）
✅ 測試集已存：predict_data/short/biz_BKK.csv（共 80 筆）
✅ 測試集已存：predict_data/short/biz_GMP.csv（共 7 筆）
✅ 測試集已存：predict_data/short/biz_HKG.csv（共 140 筆）
✅ 測試集已存：predict_data/short/biz_HND.csv（共 37 筆）
✅ 測試集已存：predict_data/short/biz_ICN.csv（共 58 筆）
✅ 測試集已存：predict_data/short/biz_NRT.csv（共 98 筆）
✅ 測試集已存：predict_data/short/biz_SIN.csv（共 43 筆）
